In [ ]:
import os

import numpy as np
import pandas as pd
from himalaya.backend import set_backend
from matplotlib import pyplot as plt
from voxelwise_tutorials.viz import plot_flatmap_from_mapper

from compare_variance_residual.fmri.features import load_feature, load_brain_data
from compare_variance_residual.fmri.results import get_result_path
from compare_variance_residual.fmri.ridge import run_ridge_pipeline

In [ ]:
data_dir = "../../data"
backend = set_backend("torch_cuda")

In [ ]:
subject = 5
modality = 'reading'
low_level_feature = "moten"

In [ ]:
alphas = np.logspace(-5, 20, 26)
number_of_delays = 4
n_targets_batch = 100
n_alphas_batch = 3
n_targets_batch_refit = 50

# Load brain data

In [ ]:
mapper_path = os.path.join("../../data", 'mappers', f"subject{subject:02}_mappers.hdf")

In [ ]:
Y, n_samples_train, run_onsets, ev = load_brain_data(data_dir, subject, modality)
print(Y.shape)
print(n_samples_train)
print(Y.shape[0] - n_samples_train)

# Low level

In [ ]:
X_low, n_samples_train = load_feature(data_dir, low_level_feature)
print(X_low.shape)
print(n_samples_train)
print(X_low.shape[0] - n_samples_train)

In [ ]:
low_level_path = os.path.join(get_result_path(modality, subject), f"{low_level_feature}_scores.csv")

In [ ]:
if not os.path.exists(low_level_path):
    low_level_scores = run_ridge_pipeline(X_low, Y, n_samples_train, run_onsets, alphas,
                                          number_of_delays, n_targets_batch, n_alphas_batch,
                                          n_targets_batch_refit)
    low_level_scores.to_csv(low_level_path, index=False)
else:
    print("Loading low level")
    low_level_scores = pd.read_csv(low_level_path)

In [ ]:
corr = low_level_scores['correlation_score']
corr = np.nan_to_num(corr)
plot_flatmap_from_mapper(corr, mapper_path, with_curvature=True, vmin=0,
                         vmax=np.max(corr), colorbar_location=[0.75, 0.05, 0.2, 0.05], cmap='magma')
plt.show()